In [1]:
import os
os.environ['SPARK_HOME'] = '/usr/local/spark'
os.environ['IPYTHON']= '1'
os.environ['PYSPARK_PYTHON'] = '/opt/conda/bin/python3.7'
os.environ['PYSPARK_DRIVER_PYTHON']= 'ipython3'
os.environ['PYSPARK_DRIVER_PYTHON_OPTS']= 'notebook'
os.environ['PYSPARK_SUBMIT_ARGS'] = '--jars /home/jovyan/spark-streaming-kafka-0-8-assembly_2.11-2.4.5.jar pyspark-shell'

In [2]:
import io
import json
import pyspark
import numpy as np
from pymongo import MongoClient
from pyspark import SparkContext
from collections import OrderedDict
from pyspark.streaming import StreamingContext
from pyspark.streaming.kafka import KafkaUtils
from schema_registry.client import SchemaRegistryClient
from schema_registry.serializers import MessageSerializer

In [3]:
class Singleton:

    def _init_(self, cls):
        self._cls = cls

    def Instance(self):
        try:
            return self._instance
        except AttributeError:
            self._instance = self._cls()
            return self._instance

    def _call_(self):
        raise TypeError('Singletons must be accessed through `Instance()`.')

    def _instancecheck_(self, inst):
        return isinstance(inst, self._cls)

In [4]:
class SchemaDecoder(object):
    HOST = os.environ['SCHEMA_REGISTRY_URL']
    SERIALIZER = None
    
    @staticmethod
    def initialize():
        client = SchemaRegistryClient(SchemaDecoder.HOST)
        SchemaDecoder.SERIALIZER = MessageSerializer(client)
        return SchemaDecoder
        
    def deserialize(s):
        decoded_message = SchemaDecoder.SERIALIZER.decode_message(s)
        return decoded_message

In [5]:
def sendRecord(partition):
    global snort_label, snort_ts
    client = MongoClient(URI) 
    mydb = client["bigdata"]
    
    netDB = mydb["netflowmeter"]
    snortDB = mydb["snorqtt"]
    joinDB = mydb["ns_relation"]
    
    for line in partition:
        if line[0] == "netflowmeter":
            netDB.insert_one(line[1])
        elif line[0] == "snorqtt":
            snort_ts = line[1].get('timestamp')
            snort_label = line[1].get('alert_msg')
            snortDB.insert_one(line[1])
            
    if(snort_ts is not None):
        netDatas = netDB.find({'timestamp': snort_ts})
        for data in netDatas:
            print(data)
            data.label = snort_label
            joinDB.insert_one(data)
            
        snort_label = None
        snort_ts = None
    
    client.close()

In [6]:
URI = "mongodb://172.31.0.3:27017"
decoder = SchemaDecoder.initialize()
snort_label = None
snort_ts = None

In [7]:
sc = SparkContext(appName="SparkTAPySpark")
ssc = StreamingContext(sc, 30)
topic = ["netflowmeter", "snorqtt"]
brokers = os.environ['BROKER_SERVER']
kafkaParams = dict({"metadata.broker.list":brokers})
kafkaStream = KafkaUtils.createDirectStream (
    ssc, 
    topic, 
    kafkaParams, 
    valueDecoder=decoder.deserialize,
    messageHandler=(lambda msg : (msg.topic, msg.message))
)

In [ ]:
kafkaStream.foreachRDD(lambda rdd : rdd.foreachPartition(sendRecord))

ssc.start()
ssc.awaitTermination()